# Analytics On AWS workshop

Take your time to read through the instructions provided in this notebook.

## 학습목표
- Glue Dev Endpoints 및 SageMaker 노트북을 사용하여 대화식으로 Glue ETL 스크립트를 작성하는 방법 이해
- boto3를 사용하여 Glue API를 호출하여 Glue 관리 및 운영 활동 수행



- 이 노트북에서 우리는 다음과 같은 클래스를 사용할 것입니다. 여기에 몇 가지 중요한 클래스가 있습니다.
    - SparkContext - Spark 기능의 주 진입점. SparkContext 는 Spark 클러스터에 대한 연결을 대표하고 RDD, accumulators, broadcast variables를 만드는데 사용될 수 있습니다. 
    - GlueContext - Apache SparkSQL SQLContext 개체를 래핑하고 Apache Spark 플랫폼과 상호 작용하기 위한 메커니즘을 제공합니다.
    - boto3 - AWS의 Python SDK, AWS API 를 호출용 라이브러리
    - awsglue - AWS의 pyspark 라이브러리

#### 아래 코드 블록을 실행하세요. 🔻

In [1]:

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import boto3
import time


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1650439833671_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 데이터셋 탐색

  - AWS Glue catalog 로부터 만들어진 raw 테이블을 위한 dynamic frame 생성
  - 데이터셋에 대한 스키마 탐색
  - raw 테이블에서 행 수 카운트
  - 데이터의 샘플 보기

## Glue Dynamic Data Frames 기초

- AWS Glue 의 dynamic data frames은 강력한 데이터 구조입니다.
- 특히 다양한 유형의 열이나 필드를 처리할 때 기본 반구조화된 데이터의 정확한 표현을 제공합니다.
- 또한 중첩 및 중첩 해제를 처리하는 강력한 기본 요소를 제공합니다.
- 동적 레코드는 자체 설명 레코드입니다. 각 레코드는 열과 유형을 인코딩하므로 모든 레코드는 동적 프레임의 다른 모든 레코드와 고유한 스키마를 가질 수 있습니다.
- ETL의 경우 더 동적인 것이 필요했기 때문에 Glue Dynamic DataFrames를 만들었습니다. DDF는 엄격한 스키마를 갖는 요구 사항을 완화하는 DF의 구현입니다. 반구조화된 데이터용으로 설계되었습니다.
- 재구성, 태그 지정 및 수정이 용이한 레코드당 스키마를 유지 관리합니다.

## DynamicFrame Class 참고
Apache Spark의 주요 추상화 중 하나는 R 및 Pandas에서 발견되는 DataFrame 구성과 유사한 SparkSQL DataFrame입니다. <br />
DataFrame은 테이블과 유사하며 기능 스타일(map/reduce/filter 등) 작업 및 SQL 작업(선택, 프로젝트, 집계)을 지원합니다. <br />

DataFrame은 강력하고 널리 사용되지만 ETL(추출, 변환 및 로드) 작업과 관련하여 제한이 있습니다. <br />
가장 중요한 것은 데이터가 로드되기 전에 스키마를 지정해야 한다는 것입니다. <br />
SparkSQL은 데이터에 대해 두 번 전달하여 이 문제를 해결합니다. <br /> 
첫 번째는 스키마를 추론하고 두 번째는 데이터를 로드합니다. <br />
그러나 이 추론은 제한적이며 지저분한 데이터의 현실을 다루지 않습니다. 예를 들어, 동일한 필드가 다른 레코드에서 다른 유형일 수 있습니다. <br />
이런 경우 Apache Spark는 종종 포기하고 원래 필드 텍스트를 사용하여 유형을 문자열로 보고합니다. <br />
이것은 올바르지 않을 수 있으며 스키마 불일치를 해결하는 방법을 더 세밀하게 제어해야 할 수 있습니다. <br />
그리고 대규모 데이터 세트의 경우 소스 데이터에 대한 추가 전달이 엄청나게 비쌀 수 있습니다. <br />

이러한 제한 사항을 해결하기 위해 AWS Glue는 DynamicFrame을 도입합니다. <br />
DynamicFrame은 각 레코드가 자체 설명적이라는 점을 제외하고 DataFrame과 유사하므로 처음에는 스키마가 필요하지 않습니다. <br />
대신 AWS Glue는 필요할 때 즉시 스키마를 계산하고 선택(또는 통합) 유형을 사용하여 스키마 불일치를 명시적으로 인코딩합니다. <br />
이러한 불일치를 해결하여 고정 스키마가 필요한 데이터 저장소와 호환되는 데이터 세트를 만들 수 있습니다.

마찬가지로 DynamicRecord는 DynamicFrame 내의 논리적 레코드를 나타냅니다. <br />
이는 자체 설명적이며 고정된 스키마를 준수하지 않는 데이터에 사용할 수 있다는 점을 제외하면 Spark DataFrame의 행과 같습니다.

스키마 불일치를 해결한 후 DynamicFrames를 DataFrames로 또는 역으로 변환할 수 있습니다.

#### DDF에 대한 추가 설명 : https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-dynamic-frame.html

#### 아래 코드 블록을 실행하세요. 🔻


In [2]:

glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Glue catalog 로부터 Dynamic Frame 생성
- 아래 블록에서 우리는 새로운 glue catalog 로부터 Dynamic Frame을 생성하기 위해 gluecontext를 사용하고 있습니다.

 Glue에서 dynamicframes을 생성하는 다른 방법
- create_dynamic_frame_from_rdd
- create_dynamic_frame_from_catalog
- create_dynamic_frame_from_options

#### dynamicframes 생성 관련 추가 리소스 https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-glue-context.html

#### 아래 코드 블록을 실행하세요. 🔻


In [3]:
raw_data = glueContext.create_dynamic_frame.from_catalog(database="analyticsworkshopdb", table_name="raw")

reference_data = glueContext.create_dynamic_frame.from_catalog(database="analyticsworkshopdb", table_name="reference_data")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# View schema
- 이 단계에서 dynamicframe 의 스키마를 봅니다.
- printSchema( ) 함수는 기반 dataframe의 스키마를 프린트합니다.

#### 아래 코드 블록을 실행하세요. 🔻

In [4]:
raw_data.printSchema()
reference_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- uuid: string
|-- device_ts: string
|-- device_id: int
|-- device_temp: int
|-- track_id: int
|-- activity_type: string
|-- partition_0: string
|-- partition_1: string
|-- partition_2: string
|-- partition_3: string

root
|-- track_id: string
|-- track_name: string
|-- artist_name: string

# 레코드의 수 카운트 하기
- dataframe에 있는 레코드의 수를 카운트합니다.
- count( ) 함수는 기반 dataframe에 있는 행수를 카운트 합니다.

#### 아래 코드 블록을 실행하세요. 🔻

In [5]:
print(f'raw_data (count) = {raw_data.count()}')
print(f'reference_data (count) = {reference_data.count()}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

raw_data (count) = 128000
reference_data (count) = 100

# 샘플 레코드 보기
- toDF 메소드를 사용하여 데이터 세트의 데이터 샘플을 표시할 수 있습니다.
- show() 메서드를 사용하여 프레임에 레코드 샘플을 표시합니다.
- 여기에 DF의 상위 5개 레코드가 표시됩니다.

#### 아래 코드 블록을 실행하세요. 🔻

In [6]:
raw_data.toDF().show(5)
reference_data.toDF().show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+---------+-----------+--------+-------------+-----------+-----------+-----------+-----------+
|                uuid|           device_ts|device_id|device_temp|track_id|activity_type|partition_0|partition_1|partition_2|partition_3|
+--------------------+--------------------+---------+-----------+--------+-------------+-----------+-----------+-----------+-----------+
|ab1a18f5-d3b6-472...|2022-04-20 07:05:...|       15|         34|      24|    Traveling|       2022|         04|         20|         07|
|b9d99888-778c-487...|2022-04-20 07:05:...|       46|         34|      15|    Traveling|       2022|         04|         20|         07|
|629d68dd-cddf-4e8...|2022-04-20 07:05:...|       32|         28|      30|    Traveling|       2022|         04|         20|         07|
|e2ef17f5-591a-4ad...|2022-04-20 07:05:...|       16|         34|      12|    Traveling|       2022|         04|         20|         07|
|dcf3b7ec-252d-482...|2022-04-20 07:05:..

# 데이터 탐색을 위해 Spark SQL 사용

- Glue 에서 데이터에 대한 SQL쿼리를 수행하기 위해 Spark SQL 엔진을 사용할 수 있습니다. 
- my_dynamic_frame 이라는 DynamicFrame 을 가지고 있다면 , Dynamic Frame을 Data Frame으로 변환하고 SQL 쿼리를 수행하기 위해 아래 코드를 사용할 수 있습니다.

### Spark SQL - Filtering & Counting - activity_type = Running
- 아래 코드 블럭에서 activity_type = Running 인 이벤트의 수를 필터링하고 카운트 할 수 있습니다.

#### 아래 코드 블록을 실행하세요. 🔻

In [7]:
# 임시테이블로써 raw_data를 추가 

raw_data.toDF().createOrReplaceTempView("temp_raw_data")

# SQL 질의
runningDF = spark.sql("select * from temp_raw_data where activity_type = 'Running'")
print(f'Running (count): {runningDF.count()}')

runningDF.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Running (count): 12725
+--------------------+--------------------+---------+-----------+--------+-------------+-----------+-----------+-----------+-----------+
|                uuid|           device_ts|device_id|device_temp|track_id|activity_type|partition_0|partition_1|partition_2|partition_3|
+--------------------+--------------------+---------+-----------+--------+-------------+-----------+-----------+-----------+-----------+
|5108e4b3-6aeb-4ee...|2022-04-20 07:05:...|       42|         32|      25|      Running|       2022|         04|         20|         07|
|fc9d2580-374d-478...|2022-04-20 07:05:...|       40|         40|      20|      Running|       2022|         04|         20|         07|
|bb614deb-349e-46c...|2022-04-20 07:05:...|       25|         28|      13|      Running|       2022|         04|         20|         07|
|2b00d93a-6d76-499...|2022-04-20 07:05:...|       43|         40|      19|      Running|       2022|         04|         20|         07|
|fc2289e5-c91b-468

### Spark SQL - Filtering & Counting - activity_type = Working
- 아래 코드 블럭에서 activity_type = Working 인 이벤트의 수를 필터링하고 카운트 할 수 있습니다. 

#### 아래 코드 블록을 실행하세요. 🔻

In [8]:
# Running the SQL statement which 
workingDF = spark.sql("select * from temp_raw_data where activity_type = 'Working'")
print(f'Working (count): {workingDF.count()}')

workingDF.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Working (count): 25692
+--------------------+--------------------+---------+-----------+--------+-------------+-----------+-----------+-----------+-----------+
|                uuid|           device_ts|device_id|device_temp|track_id|activity_type|partition_0|partition_1|partition_2|partition_3|
+--------------------+--------------------+---------+-----------+--------+-------------+-----------+-----------+-----------+-----------+
|ca8a1696-de82-4dd...|2022-04-20 07:05:...|       46|         32|      16|      Working|       2022|         04|         20|         07|
|1cfe4c56-3d25-4d4...|2022-04-20 07:05:...|       28|         32|      21|      Working|       2022|         04|         20|         07|
|e0557d3d-76ab-4cd...|2022-04-20 07:05:...|       49|         32|      11|      Working|       2022|         04|         20|         07|
|8ed86736-fef6-45b...|2022-04-20 07:05:...|       40|         34|      12|      Working|       2022|         04|         20|         07|
|da91e51f-ec66-48d

### Glue Transforms - Filtering & Counting - activity_type = Running
- 이제 Glue 내장 변환을 사용하여 동일한 작업을 수행할 수 있습니다.
- **filter** 변환을 사용합니다.
- Filter()함수는 DynamicFrame에서 레코드를 선택하고 필터링된 DynamicFrame을 반환합니다.
- 레코드가 출력되는지(함수가 true를 반환하는지) 아닌지 (함수가 false를 반환)를 결정하는 함수를 지정합니다.
- 이 함수에서는 activity_type == 'Running' 조건을 필터링합니다.

#### Read More: https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-transforms-filter.html#aws-glue-api-crawler-pyspark-transforms-filter-example

#### 아래 코드 블록을 실행하세요. 🔻

In [9]:

def filter_function(dynamic_record):
	if dynamic_record['activity_type'] == 'Running':
		return True
	else:
		return False
runningDF = Filter.apply(frame=raw_data, f=filter_function)

print(f'Running (count): {runningDF.count()}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Running (count): 12725

### Glue Transforms - Filtering & Counting - activity_type = Working (Using python Lambda Expressions)
- python lambda 키워드로 작은 익명 함수를 만들 수 있습니다.
- python lambda 함수는 함수 객체가 필요한 모든 곳에서 사용할 수 있습니다. 구문상 단일 표현식으로 제한됩니다.
- 예: 이 함수는 두 개의 인수인 a + b의 합계를 반환합니다. lambda a,b : a + b
- 아래 예에서는 x 를 인자로 받아 x 의 activity_type 이 Working 인 것을 반환합니다.

#### 아래 코드 블록을 실행하세요. 🔻

In [10]:

workingDF = Filter.apply(frame=raw_data, f=lambda x: x['activity_type'] == 'Working')

print(f'Working (count): {workingDF.count()}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Working (count): 25692

### Glue Transforms - 두개의 dataframes 결합
- 두 개의 DynamicFrames에서 동등 조인을 수행합니다.
- 이 변환은 다음 인수를 허용합니다.
     - frame1 – 조인할 첫 번째 DynamicFrame
     - frame2 – 결합할 두 번째 DynamicFrame
     - keys1 – 첫 번째 프레임에 결합할 키
     - keys2 – 두 번째 프레임에 결합할 키
- 우리의 경우 **raw_data** & **reference_data** 두 프레임을 결합합니다.
- **track_id** 열을 기반으로 이 두 프레임을 결합합니다.

#### Read More: https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-transforms-join.html

#### 아래 코드 블록을 실행하세요. 🔻

In [11]:

joined_data = Join.apply(raw_data, reference_data, 'track_id', 'track_id')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### schema 보기
- 이 단계에서 dynamic frame 의 스키마를 봅니다.
- printSchema( ) 함수는 기반 DataFrame 의 스키마를 프린트합니다.

#### 아래 코드 블록을 실행하세요. 🔻

In [12]:
joined_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- track_id: string
|-- partition_2: string
|-- activity_type: string
|-- .track_id: int
|-- partition_1: string
|-- device_temp: int
|-- track_name: string
|-- artist_name: string
|-- partition_3: string
|-- device_ts: string
|-- device_id: int
|-- partition_0: string
|-- uuid: string

###### Cleaning up the joined_data dynamicframe
- 관심 있는 열 외에 파티션 열이 있습니다.
- 이들은 S3의 yyyy/mm/dd/hh 디렉토리 구조에 파일을 배치하기 위해 Firehose에 의해 생성되었습니다.
- Glue의 내장 **DropFields** 변환을 사용하여 파티션 열을 삭제합니다.

###### Read more about AWS Glue transforms here : https://docs.aws.amazon.com/glue/latest/dg/built-in-transforms.html

#### 아래 코드 블록을 실행하세요. 🔻

In [13]:

joined_data_clean = DropFields.apply(frame=joined_data, paths=['partition_0','partition_1','partition_2','partition_3'])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### DropFields 변환후에 스키마 보기
- printSchema 함수로 스키마를 보고
- show 함수로 샘플 데이터를 봅니다.

#### 아래 코드 블록을 실행하세요. 🔻

In [21]:
joined_data_clean.printSchema()
joined_data_clean.toDF().show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- track_id: string
|-- activity_type: string
|-- .track_id: int
|-- device_temp: int
|-- track_name: string
|-- artist_name: string
|-- device_ts: string
|-- device_id: int
|-- uuid: string

+--------+-------------+---------+-----------+----------+-----------+--------------------+---------+--------------------+
|track_id|activity_type|.track_id|device_temp|track_name|artist_name|           device_ts|device_id|                uuid|
+--------+-------------+---------+-----------+----------+-----------+--------------------+---------+--------------------+
|       1|    Traveling|        1|         28|God's Plan|      Drake|2022-04-20 07:05:...|       24|90d06018-e4ab-41b...|
|       1|      Working|        1|         34|God's Plan|      Drake|2022-04-20 07:05:...|       50|3c42f105-feff-4f5...|
|       1|      Sitting|        1|         32|God's Plan|      Drake|2022-04-20 07:05:...|       38|9451b20e-d2e3-45e...|
|       1|      Working|        1|         34|God's Plan|      Drake|2

###### sample data 

# 변환의 마지막 단계 - 변환된 데이터를 S3 에 저장
- 이 단계에서는 Glue의 write_dynamic_frame 기능을 사용하여 변환된 데이터를 S3에 씁니다.
- 변환된 데이터를 다른 디렉토리에 parquet 형식으로 저장합니다.
- 버킷 이름을 반영하도록 D3 버킷 이름 **yourname-analytics-workshop-bucket**을 변경해야 합니다.


---
- 왜 parquet 형식입니까?
    - Apache Parquet은 데이터의 빠른 검색에 최적화되어 있으며 AWS 분석 애플리케이션에서 사용되는 열 기반 스토리지 형식입니다.
    - 열 기반 스토리지 형식에는 Athena와 함께 사용하기에 적합한 다음과 같은 특성이 있습니다.
      Amazon S3의 스토리지 공간을 절약하고 쿼리 처리 중 디스크 공간과 I/O를 줄이기 위해 열 데이터 유형에 대해 압축 알고리즘이 선택된 열별 압축.
    - Parquet 및 ORC의 조건자 푸시다운을 통해 쿼리에서 필요한 블록만 가져올 수 있어 쿼리 성능이 향상됩니다.
    - 쿼리가 데이터에서 특정 열 값을 가져올 때 최대/최소 값과 같은 데이터 블록 조건자의 통계를 사용하여 블록을 읽을지 건너뛸지를 결정합니다.
    - Parquet에서 데이터 분할을 통해 분석 도구는 데이터 읽기를 여러 판독기(reader) 로 분할하고 쿼리 처리 중에 병렬성을 높일 수 있습니다.
    
#### 아래 코드 블록을 실행하세요. 🔻

In [20]:
try:
    datasink = glueContext.write_dynamic_frame.from_options(
        frame = joined_data_clean, connection_type="s3",
        connection_options = {"path": "s3://juhwan-analytics-workshop-bucket/data/processed-data/"},
        format = "parquet")
    print('Transformed data written to S3')
except Exception as ex:
    print('Something went wrong')
    print(ex)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Transformed data written to S3

# AWS Glue 를 수행하고 자동화하기 위해 boto3 라이브러리 사용

- Boto는 Python용 AWS SDK입니다. 이를 통해 Python 개발자는 AWS 서비스를 생성, 구성 및 관리할 수 있습니다. Boto는 사용하기 쉬운 객체 지향 API와 AWS 서비스에 대한 저수준 액세스를 제공합니다.


# 변환된 데이터넷을 glue catalog 에 추가 
- 이제 변환된 데이터를 S3에 작성했으므로 Athena 를 사용하여 쿼리할 수 있도록 Glue Catalog 에 추가해야 합니다.
- 이 블록을 실행하는 데 60초 가까이 걸리며 종료하거나 실행을 중지하지 않도록 합니다.

#### 아래 코드 블록을 실행하세요. 🔻

In [25]:

glueclient = boto3.client('glue', region_name='us-east-1')

response = glueclient.start_crawler(Name='AnalyticsworkshopCrawler')

print('---')

crawler_state = None
while (crawler_state != 'STOPPING'):
    response = glueclient.get_crawler(Name='AnalyticsworkshopCrawler')
    crawler_state = str(response['Crawler']['State'])
    time.sleep(1)

print('Crawler Stopped')
print('---')
time.sleep(3)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
Crawler Stopped
---

# analyticsworkshopdb 데이터베이스에 있는 테이블의 리스트를 보기 위해 boto 를 사용합니다. 

#### 아래 코드 블록을 실행하세요 🔻

In [26]:

print('** analyticsworkshopdb has following tables**')
response = glueclient.get_tables(
    DatabaseName='analyticsworkshopdb',
)

for table in response['TableList']:
    print(table['Name'])



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

** analyticsworkshopdb has following tables**
processed_data
raw
reference_data

# 😎
=========================

### 이 노트북과 그 결과물을 집으로 가져가고 싶다면 다운로드/내보낼 수 있습니다.

- Jupyter의 메뉴 표시줄에서 **파일**을 클릭합니다.
     - 다른 이름으로 다운로드: 노트북(.ipynb) (향후 jupyter 노트북으로 다시 가져올 수 있음)
     - 다른 이름으로 다운로드: HTML(읽기 쉬운 형식으로 코드 + 결과 표시)


# NEXT Steps: lab guide 로 돌아갑니다. 

=========================